In [29]:
%env OGDF_BUILD_DIR=~/OGDF-master/build-debug
from ogdf_python import ogdf, cppinclude
cppinclude("ogdf/basic/graph_generators/randomized.h")
cppinclude("ogdf/layered/SugiyamaLayout.h")
cppinclude("ogdf/basic/graph_generators/deterministic.h")
cppinclude("ogdf/energybased/FMMMLayout.h")


G = ogdf.Graph()
ogdf.setSeed(1)
ogdf.gridGraph(G, 3, 3, True,True)
GA = ogdf.GraphAttributes(G, ogdf.GraphAttributes.all)
GA.destroyAttributes(ogdf.GraphAttributes.nodeId)

for n in G.nodes:
    GA.label[n] = "N%s" % n.index()
    GA.x[n] = (n.index() %3) * 50 
    GA.y[n] = (n.index()//3) * 50
    
#or adj in G.nodes[0].adjEntries:
#   print(adj.twinNode().index())
#   if adj.twinNode().index() == 9:
#       e = adj.theEdge()
#       GA.bends[e].pushBack(ogdf.DPoint(-100,-100)) 
    
#SL = ogdf.SugiyamaLayout()
#SL=ogdf.FMMMLayout();
 
#SL.useHighLevelOptions(True);
#SL.unitEdgeLength(15.0);
#SL.newInitialPlacement(True);
#SL.qualityVersusSpeed(ogdf.FMMMOptions.QualityVsSpeed.GorgeousAndEfficient);
#SL.call(GA)
ogdf.GraphIO.write(GA, "sugiyama-simple.graphml")

GA

env: OGDF_BUILD_DIR=~/OGDF-master/build-debug


In [ ]:
%env OGDF_BUILD_DIR=~/OGDF-master/build-debug
from ogdf_python import ogdf, cppinclude
cppinclude("ogdf/basic/graph_generators/randomized.h")
cppinclude("ogdf/layered/SugiyamaLayout.h")
cppinclude("ogdf/basic/graph_generators/deterministic.h")
cppinclude("ogdf/energybased/FMMMLayout.h")
cppinclude("ogdf/labeling/ELabelInterface.h")


# Small, random biconnected graph 
G = ogdf.Graph()
ogdf.setSeed(1)
ogdf.randomBiconnectedGraph(G, 4, 4)
GA = ogdf.GraphAttributes(G, ogdf.GraphAttributes.all)

#label the edges as being horizontal or vertical and whether it wraps around (this can be obtained from edge length)
for i,e in enumerate(G.edges):
    
    if i == 0:
        GA.label[e] = "v_around"
    elif i == 1:
        GA.label[e] = "h_inside"
    elif i == 2:
        GA.label[e] = "v_inside"
    else:
        GA.label[e] = "h_around"
  
#assign coordinates to each node
for i,n in enumerate(G.nodes):
    GA.label[n] = "N%s" % i
    GA.width[n] = 10
    GA.height[n] = 10
    if i == 0:
        GA.x[n] =0 
        GA.y[n] =0
    elif i == 1:
        GA.x[n] = 0
        GA.y[n] = 50
    elif i == 2:
        GA.x[n] = 50
        GA.y[n] = 50
    else:
        GA.x[n] = 50
        GA.y[n] = 0


done = []
for i,n in enumerate(G.nodes):
    
    for i, adj in enumerate(n.adjEntries):
        
        e = adj.theEdge()
        v = adj.twinNode()
        n_x = GA.x[n]
        n_y = GA.y[n]
        v_x = GA.x[v]
        v_y = GA.y[v]
        
        if e in done:
            continue
            
        if GA.label[e] == "v_around": 
            #if v_y > n_y: 
            
            GA.bends[e].pushBack(ogdf.DPoint(n_x,n_y-100))
            GA.bends[e].pushBack(ogdf.DPoint(n_x-100, n_y-100))
            GA.bends[e].pushBack(ogdf.DPoint(n_x-100,v_y+100))
            GA.bends[e].pushBack(ogdf.DPoint(v_x,v_y+100))
                
        elif GA.label[e] == 'h_around':
            #if n_x > v_x:
                
            GA.bends[e].pushBack(ogdf.DPoint(n_x+100,n_y))
            GA.bends[e].pushBack(ogdf.DPoint(n_x+100, v_y+100))
            GA.bends[e].pushBack(ogdf.DPoint(v_x-100,v_y+100))
            GA.bends[e].pushBack(ogdf.DPoint(v_x-100,v_y))
        
        done.append(e)
    

GA     

In [1]:
%env OGDF_BUILD_DIR=~/OGDF-master/build-debug
from ogdf_python import ogdf, cppinclude
cppinclude("ogdf/basic/graph_generators/deterministic.h")

# 0 -- > x
# |
# v
#  y

def add_bends(line, *points):
    for point in points:
        line.pushBack(ogdf.DPoint(*point))

def find_wrap(GA, e):
    s,t = e.source(), e.target()
    wrap_h = GA.x[t] < GA.x[s]
    wrap_v = GA.y[t] < GA.y[s]
    if wrap_h and wrap_v:
        delta_x = abs(GA.x[t] - GA.x[s])
        delta_y = abs(GA.y[t] - GA.y[s])
        if delta_x > delta_y:
            wrap = "hv"
        else:
            wrap = "vh"
    elif wrap_h:
        wrap = "h"
    elif wrap_v:
        wrap = "v"
    else:
        wrap = ""
    return wrap

def apply_wrap(GA, box, e, wrap, margin=50):
    if not wrap:
        return
    
    s,t = e.source(), e.target()
    left = box.p1().m_x
    top = box.p1().m_y
    right = box.p2().m_x
    bottom = box.p2().m_y
    
    GA.bends[e].clear()
    if wrap[0] == "h":
        add_bends(GA.bends[e],
            (right + margin, GA.y[s]),
            (right + margin, top - margin),
            (left - margin, top - margin),
            (left - margin, GA.y[t]),
        )
    elif wrap[0] == "v":
        add_bends(GA.bends[e],
            (GA.x[s], bottom + margin),
            (left - margin, bottom + margin),
            (left - margin, top - margin),
            (GA.x[t], top - margin),
        )


def layout_edges(GA, box):
    wraps = ogdf.EdgeArray[str](GA.constGraph(), "")
    wraps_h = []
    wraps_v = []
    for e in GA.constGraph().edges:
        w = wraps[e] = find_wrap(GA, e)
        if w and w[0] == "h":
            wraps_h.append(e)
        elif w and w[0] == "v":
            wraps_v.append(e)
    wraps_h.sort(key=lambda e: GA.y[e.source()])
    wraps_v.sort(key=lambda e: GA.x[e.source()])
    
    for nr, e in enumerate(wraps_h + wraps_v):
        # GA.arrowType[e] = getattr(ogdf.EdgeArrow, "None")
        apply_wrap(GA, box, e, wraps[e], 50 + nr * 5)
        if wraps[e] in ["hv", "vh"]:
            GA.strokeColor[e] = ogdf.Color(0,255,0)
        elif wraps[e] == "h":
            GA.strokeColor[e] = ogdf.Color(255,0,0)
        elif wraps[e] == "v":
            GA.strokeColor[e] = ogdf.Color(0,0,255)
        
    return GA

def draw_bounding_box(G, GA, box):
    box_node1 = G.newNode()
    box_node2 = G.newNode()
    GA.x[box_node1] = box.p1().m_x - 25
    GA.y[box_node1] = box.p1().m_y - 25
    GA.x[box_node2] = box.p2().m_x + 25
    GA.y[box_node2] = box.p2().m_y + 25
    GA.width[box_node1] = 0
    GA.height[box_node1] = 0
    GA.width[box_node2] = 0
    GA.height[box_node2] = 0
    box_edge1 = G.newEdge(box_node1, box_node2)
    box_edge2 = G.newEdge(box_node1, box_node2)
    add_bends(GA.bends[box_edge1], (box.p1().m_x - 25, box.p2().m_y + 25))
    add_bends(GA.bends[box_edge2], (box.p2().m_x + 25, box.p1().m_y - 25))

env: OGDF_BUILD_DIR=~/OGDF-master/build-debug


In [2]:
width = 4
height = 4

G = ogdf.Graph()
ogdf.gridGraph(G, width, height, True, True)
GA = ogdf.GraphAttributes(G, ogdf.GraphAttributes.all)
GA.destroyAttributes(ogdf.GraphAttributes.nodeId)

for n in G.nodes:
    GA.label[n] = str(n.index())
    GA.x[n] = (n.index() % width) * 50 
    GA.y[n] = (n.index() // height) * 50

box = GA.boundingBox()
layout_edges(GA, box)
# draw_bounding_box(G, GA, box)
GA

In [3]:
G.delNode(G.nodes[0])
G.delNode(G.nodes[1])
G.delNode(G.nodes[2])
G.delNode(G.nodes[4])
G.delNode(G.nodes[8])
G.delNode(G.nodes[12])
G.delNode(G.nodes[13])
G.delNode(G.nodes[11])
G.delNode(G.nodes[14])


G.delEdge(G.searchEdge(G.nodes[5], G.nodes[6]))

#G.unsplit(G.nodes[1]) # replace deg 2 node by edge

G.newEdge(G.nodes[9],G.nodes[5])
G.newEdge(G.nodes[3],G.nodes[5])
G.newEdge(G.nodes[10],G.nodes[6])
G.newEdge(G.nodes[7],G.nodes[15])



layout_edges(GA, box)

In [89]:
def adjList(nodes):
    
    adjList = {}
    
    for n in G.nodes:
        
        #adjList[GA.label[n]] = {}
        adjList[n] = {}
        
        for adj in n.adjEntries:
            
            e = adj.theEdge()
            wrap = find_wrap(GA, e)
            
            if n == e.source():
                
                if wrap == "h" or wrap == "hv":
                    position = 1
                    
                elif wrap == "v" or wrap == "vh":
                    position = 2
                    
                else:
                    if GA.x[n] == GA.x[adj.twinNode()]:
                        position = 2
                        
                    if GA.y[n] == GA.y[adj.twinNode()]:
                        position = 1
        
                #adjList[GA.label[n]][position] = GA.label[adj.twinNode()]  #uncomment to use node lab
                adjList[n][position] = adj
                        
            else:
                
                if wrap == "h" or wrap == "hv":
                    position = 3
                    
                elif wrap == "v" or wrap == "vh":
                    position = 0
                    
                else:
                    if GA.x[n] == GA.x[adj.twinNode()]:
                        position = 0
                        
                    if GA.y[n] == GA.y[adj.twinNode()]:
                        position = 3
                        
                #adjList[GA.label[n]][position] = GA.label[adj.twinNode()]  #uncomment to use node labels
                adjList[n][position] = adj
                
    
    return adjList

result = adjList(G.nodes)

result

{<cppyy.gbl.ogdf.NodeElement object at 0x60cf890>: {2: <cppyy.gbl.ogdf.AdjElement object at 0x61b5830>,
  0: <cppyy.gbl.ogdf.AdjElement object at 0x61b5740>,
  1: <cppyy.gbl.ogdf.AdjElement object at 0x61b5dd0>},
 <cppyy.gbl.ogdf.NodeElement object at 0x60cf950>: {2: <cppyy.gbl.ogdf.AdjElement object at 0x61b5d40>,
  0: <cppyy.gbl.ogdf.AdjElement object at 0x61b5500>,
  3: <cppyy.gbl.ogdf.AdjElement object at 0x61b5e00>},
 <cppyy.gbl.ogdf.NodeElement object at 0x60cfa50>: {1: <cppyy.gbl.ogdf.AdjElement object at 0x61b53b0>,
  2: <cppyy.gbl.ogdf.AdjElement object at 0x61b5aa0>,
  0: <cppyy.gbl.ogdf.AdjElement object at 0x61b5890>},
 <cppyy.gbl.ogdf.NodeElement object at 0x60cfa90>: {3: <cppyy.gbl.ogdf.AdjElement object at 0x61b5110>,
  0: <cppyy.gbl.ogdf.AdjElement object at 0x61b5230>,
  2: <cppyy.gbl.ogdf.AdjElement object at 0x61b5380>},
 <cppyy.gbl.ogdf.NodeElement object at 0x60cfc10>: {0: <cppyy.gbl.ogdf.AdjElement object at 0x61b5d10>,
  1: <cppyy.gbl.ogdf.AdjElement object at 0x

In [90]:
def sorted_adjList(adjList):
    
    sorted_adjList = {}
    
    for n in adjList:
        
        sortedList = []
        
        for i in sorted (adjList[n]) :
            
            sortedList.append(adjList[n][i])
        
        sorted_adjList[n] = sortedList
    
    return sorted_adjList
    
    
sorted_adjList = sorted_adjList(result)
sorted_adjList

{<cppyy.gbl.ogdf.NodeElement object at 0x60cf890>: [<cppyy.gbl.ogdf.AdjElement object at 0x61b5740>,
 <cppyy.gbl.ogdf.NodeElement object at 0x60cf950>: [<cppyy.gbl.ogdf.AdjElement object at 0x61b5500>,
 <cppyy.gbl.ogdf.NodeElement object at 0x60cfa50>: [<cppyy.gbl.ogdf.AdjElement object at 0x61b5890>,
 <cppyy.gbl.ogdf.NodeElement object at 0x60cfa90>: [<cppyy.gbl.ogdf.AdjElement object at 0x61b5230>,
 <cppyy.gbl.ogdf.NodeElement object at 0x60cfc10>: [<cppyy.gbl.ogdf.AdjElement object at 0x61b5d10>,
 <cppyy.gbl.ogdf.NodeElement object at 0x60cfbd0>: [<cppyy.gbl.ogdf.AdjElement object at 0x61b5a70>,
 <cppyy.gbl.ogdf.NodeElement object at 0x60cf990>: [<cppyy.gbl.ogdf.AdjElement object at 0x61b5350>,
  <cppyy.gbl.ogdf.AdjElement object at 0x61b5710>]}

In [95]:

G.sort(list(sorted_adjList)[0],sorted_adjList[list(sorted_adjList)[0]] )

IncrementalExecutor::executeFunction: symbol '_ZN4ogdf8internal13GraphListBase4sortISt6vectorIPNS_10AdjElementESaIS5_EEEEvRKT_' unresolved while linking symbol '__cf_82'!
You are probably missing the definition of void ogdf::internal::GraphListBase::sort<std::vector<ogdf::AdjElement*, std::allocator<ogdf::AdjElement*> > >(std::vector<ogdf::AdjElement*, std::allocator<ogdf::AdjElement*> > const&)
Maybe you need to load the corresponding shared library?


In [91]:
sorted_adjList[list(sorted_adjList)[0]]